# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [59]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df=nhl_df[nhl_df["year"]==2018].dropna() #clearing nan values and checking year==2018
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #we need to create dictionary with areas names (for merging cities and nhl_df)
    tmp_dict = {'Tampa Bay Lightning*': 'Tampa Bay Area',
                      'Boston Bruins*': 'Boston',
                      'Toronto Maple Leafs*': 'Toronto',
                      'Florida Panthers': 'Miami–Fort Lauderdale',
                      'Detroit Red Wings': 'Detroit',
                      'Montreal Canadiens': 'Montreal',
                      'Ottawa Senators': 'Ottawa',
                      'Buffalo Sabres': 'Buffalo',
                      'Washington Capitals*': 'Washington, D.C.',
                      'Pittsburgh Penguins*': 'Pittsburgh',
                      'Philadelphia Flyers*': 'Philadelphia',
                      'Columbus Blue Jackets*': 'Columbus',
                      'New Jersey Devils*': 'New York City',
                      'Carolina Hurricanes': 'Raleigh',
                      'New York Islanders': 'New York City',
                      'New York Rangers': 'New York City',
                      'Nashville Predators*': 'Nashville',
                      'Winnipeg Jets*': 'Winnipeg',
                      'Minnesota Wild*': 'Minneapolis–Saint Paul',
                      'Colorado Avalanche*': 'Denver',
                      'St. Louis Blues': 'St. Louis',
                      'Dallas Stars': 'Dallas–Fort Worth',
                      'Chicago Blackhawks': 'Chicago',
                      'Vegas Golden Knights*': 'Las Vegas',
                      'Anaheim Ducks*': 'Los Angeles',
                      'San Jose Sharks*': 'San Francisco Bay Area',
                      'Los Angeles Kings*': 'Los Angeles',
                      'Calgary Flames': 'Calgary',
                      'Edmonton Oilers': 'Edmonton',
                      'Vancouver Canucks': 'Vancouver',
                      'Arizona Coyotes': 'Phoenix'}
    #clearing raw rows
    nhl_df=nhl_df[nhl_df.apply(lambda x: x.values.tolist() not
                               in nhl_df[nhl_df["W"].str.contains(r"[A-Za-z]")].values.tolist(), axis=1)]
    nhl_df=nhl_df.astype({"W": int,"L":int})
    
    nhl_df['WL_Ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    nhl_df['Metropolitan area']= nhl_df['team'].map(tmp_dict)
    #group
    win_loss_by_region = nhl_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    cities=cities[~cities["NHL"].str.contains("—")&~cities["NHL"].str.contains("^\[")]
    win_loss_by_region = pd.merge(win_loss_by_region, cities, how = 'left', left_index = True, right_index = True)
    win_loss_by_region['Population'] = win_loss_by_region['Population'].astype(int)
    population_by_region = win_loss_by_region['Population']
    win_loss_by_region = win_loss_by_region['WL_Ratio']
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [58]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nba_df=nba_df[nba_df["year"]==2018].dropna()
    nba_df["team"]=nba_df["team"].str.replace("\(\d+\)","",regex=True)
    nba_df["team"]=nba_df["team"].str.replace("\xa0","",regex=True)
    nba_df["team"]=nba_df["team"].str.replace("*","",regex=True)
    tmp_dict={}
    tmp_lst=["Toronto","Boston","Philadelphia",'Cleveland','Indianapolis','Miami–Fort Lauderdale','Milwaukee','Washington, D.C.',
    'Detroit','Charlotte','New York City','New York City','Chicago','Orlando','Atlanta','Houston','San Francisco Bay Area',
    'Portland','Oklahoma City','Salt Lake City','New Orleans','San Antonio','Minneapolis–Saint Paul','Denver','Los Angeles',
    'Los Angeles','Sacramento','Dallas–Fort Worth','Memphis','Phoenix']
    r=0
    for i in nba_df["team"]:
        tmp_dict[i]=tmp_lst[r]
        r+=1
    nba_df=nba_df.astype({"W": int,"L":int})
    nba_df['WL_Ratio'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    nba_df['Metropolitan area']= nba_df['team'].map(tmp_dict)
    
    win_loss_by_region = nba_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    win_loss_by_region = pd.merge(win_loss_by_region, cities, how = 'left', left_index = True, right_index = True)
    win_loss_by_region['Population'] = win_loss_by_region['Population'].astype(int)
    population_by_region = win_loss_by_region['Population']
    win_loss_by_region = win_loss_by_region['WL_Ratio']
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation() 


-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [57]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df=mlb_df[mlb_df["year"]==2018].dropna()
    tmp_dict={}
    tmp_lst=['Boston','New York City','Tampa Bay Area','Toronto','Baltimore','Cleveland','Minneapolis–Saint Paul',
             'Detroit','Chicago','Kansas City','Houston','San Francisco Bay Area','Seattle','Los Angeles','Dallas–Fort Worth',
             'Atlanta','Washington, D.C.','Philadelphia','New York City','Miami–Fort Lauderdale','Milwaukee','Chicago',
            'St. Louis','Pittsburgh','Cincinnati','Los Angeles','Denver','Phoenix','San Francisco Bay Area','San Diego']
    r=0
    for i in mlb_df["team"]:
        tmp_dict[i]=tmp_lst[r]
        r+=1
    mlb_df=mlb_df.astype({"W": int,"L":int})
    mlb_df['WL_Ratio'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    mlb_df['Metropolitan area']= mlb_df['team'].map(tmp_dict)
    
    win_loss_by_region = mlb_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    win_loss_by_region = pd.merge(win_loss_by_region, cities, how = 'left', left_index = True, right_index = True)
    win_loss_by_region['Population'] = win_loss_by_region['Population'].astype(int)
    population_by_region = win_loss_by_region['Population']
    win_loss_by_region = win_loss_by_region['WL_Ratio']
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [56]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df=nfl_df[nfl_df["year"]==2018].dropna()
    nfl_df['team'] = nfl_df['team'].str.replace('*', '', regex = True)
    nfl_df['team'] = nfl_df['team'].str.replace('+', '', regex = True)
    nfl_df=nfl_df[nfl_df.apply(lambda x: x.values.tolist() not
                               in nfl_df[nfl_df["W"].str.contains(r"[A-Za-z]")].values.tolist(), axis=1)]
    nfl_dct = {'New England Patriots': 'Boston',
                      'Miami Dolphins': 'Miami–Fort Lauderdale',
                      'Buffalo Bills': 'Buffalo',
                      'New York Jets': 'New York City',
                      'Baltimore Ravens': 'Baltimore',
                      'Pittsburgh Steelers': 'Pittsburgh',
                      'Cleveland Browns': 'Cleveland',
                      'Cincinnati Bengals': 'Cincinnati',
                      'Houston Texans': 'Houston',
                      'Indianapolis Colts': 'Indianapolis',
                      'Tennessee Titans': 'Nashville',
                      'Jacksonville Jaguars': 'Jacksonville',
                      'Kansas City Chiefs': 'Kansas City',
                      'Los Angeles Chargers': 'Los Angeles',
                      'Denver Broncos': 'Denver',
                      'Oakland Raiders': 'San Francisco Bay Area',
                      'Dallas Cowboys': 'Dallas–Fort Worth',
                      'Philadelphia Eagles': 'Philadelphia',
                      'Washington Redskins': 'Washington, D.C.',
                      'New York Giants': 'New York City',
                      'Chicago Bears': 'Chicago',
                      'Minnesota Vikings': 'Minneapolis–Saint Paul',
                      'Green Bay Packers': 'Green Bay',
                      'Detroit Lions': 'Detroit',
                      'New Orleans Saints': 'New Orleans',
                      'Carolina Panthers': 'Charlotte',
                      'Atlanta Falcons': 'Atlanta',
                      'Tampa Bay Buccaneers': 'Tampa Bay Area',
                      'Los Angeles Rams': 'Los Angeles',
                      'Seattle Seahawks': 'Seattle',
                      'San Francisco 49ers': 'San Francisco Bay Area',
                      'Arizona Cardinals': 'Phoenix'}

    nfl_df['Metropolitan area'] = nfl_df['team'].map(nfl_dct)
    nfl_df=nfl_df.astype({"W": int,"L":int})
    nfl_df['WL_Ratio'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    
    win_loss_by_region = nfl_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    win_loss_by_region = pd.merge(win_loss_by_region, cities, how = 'left', left_index = True, right_index = True)
    win_loss_by_region['Population'] = win_loss_by_region['Population'].astype(int)
    population_by_region = win_loss_by_region['Population']
    win_loss_by_region = win_loss_by_region['WL_Ratio']
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [55]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    #==========================================nhl
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df=nhl_df[nhl_df["year"]==2018].dropna() #clearing nan values and checking year==2018
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    #we need to create dictionary with areas names (for merging cities and nhl_df)
    tmp_dict = {'Tampa Bay Lightning*': 'Tampa Bay Area',
                      'Boston Bruins*': 'Boston',
                      'Toronto Maple Leafs*': 'Toronto',
                      'Florida Panthers': 'Miami–Fort Lauderdale',
                      'Detroit Red Wings': 'Detroit',
                      'Montreal Canadiens': 'Montreal',
                      'Ottawa Senators': 'Ottawa',
                      'Buffalo Sabres': 'Buffalo',
                      'Washington Capitals*': 'Washington, D.C.',
                      'Pittsburgh Penguins*': 'Pittsburgh',
                      'Philadelphia Flyers*': 'Philadelphia',
                      'Columbus Blue Jackets*': 'Columbus',
                      'New Jersey Devils*': 'New York City',
                      'Carolina Hurricanes': 'Raleigh',
                      'New York Islanders': 'New York City',
                      'New York Rangers': 'New York City',
                      'Nashville Predators*': 'Nashville',
                      'Winnipeg Jets*': 'Winnipeg',
                      'Minnesota Wild*': 'Minneapolis–Saint Paul',
                      'Colorado Avalanche*': 'Denver',
                      'St. Louis Blues': 'St. Louis',
                      'Dallas Stars': 'Dallas–Fort Worth',
                      'Chicago Blackhawks': 'Chicago',
                      'Vegas Golden Knights*': 'Las Vegas',
                      'Anaheim Ducks*': 'Los Angeles',
                      'San Jose Sharks*': 'San Francisco Bay Area',
                      'Los Angeles Kings*': 'Los Angeles',
                      'Calgary Flames': 'Calgary',
                      'Edmonton Oilers': 'Edmonton',
                      'Vancouver Canucks': 'Vancouver',
                      'Arizona Coyotes': 'Phoenix'}
    #clearing raw rows
    nhl_df=nhl_df[nhl_df.apply(lambda x: x.values.tolist() not
                               in nhl_df[nhl_df["W"].str.contains(r"[A-Za-z]")].values.tolist(), axis=1)]
    nhl_df=nhl_df.astype({"W": int,"L":int})
    
    nhl_df['WL_Ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
    nhl_df['Metropolitan area']= nhl_df['team'].map(tmp_dict)
    #group
    nhl_out = nhl_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    cities=cities[~cities["NHL"].str.contains("—")&~cities["NHL"].str.contains("^\[")]
    nhl_out = pd.merge(nhl_out, cities, how = 'left', left_index = True, right_index = True)
    nhl_out['Population'] = nhl_out['Population'].astype(int)
    nhl_out=nhl_out[['WL_Ratio', 'Population']]
    
    
     #==========================================nba
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nba_df=nba_df[nba_df["year"]==2018].dropna()
    nba_df["team"]=nba_df["team"].str.replace("\(\d+\)","",regex=True)
    nba_df["team"]=nba_df["team"].str.replace("\xa0","",regex=True)
    nba_df["team"]=nba_df["team"].str.replace("*","",regex=True)
    tmp_dict={}
    tmp_lst=["Toronto","Boston","Philadelphia",'Cleveland','Indianapolis','Miami–Fort Lauderdale','Milwaukee','Washington, D.C.',
    'Detroit','Charlotte','New York City','New York City','Chicago','Orlando','Atlanta','Houston','San Francisco Bay Area',
    'Portland','Oklahoma City','Salt Lake City','New Orleans','San Antonio','Minneapolis–Saint Paul','Denver','Los Angeles',
    'Los Angeles','Sacramento','Dallas–Fort Worth','Memphis','Phoenix']
    r=0
    for i in nba_df["team"]:
        tmp_dict[i]=tmp_lst[r]
        r+=1
    nba_df=nba_df.astype({"W": int,"L":int})
    nba_df['WL_Ratio'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    nba_df['Metropolitan area']= nba_df['team'].map(tmp_dict)
    
    win_loss_by_region = nba_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    nba_out=pd.merge(win_loss_by_region, cities, how = 'left', left_index = True, right_index = True)
    nba_out['Population'] = nba_out['Population'].astype(int)
    nba_out=nba_out[['WL_Ratio', 'Population']]
    
    
    #========================================mlb
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df=mlb_df[mlb_df["year"]==2018].dropna()
    tmp_dict={}
    tmp_lst=['Boston','New York City','Tampa Bay Area','Toronto','Baltimore','Cleveland','Minneapolis–Saint Paul',
             'Detroit','Chicago','Kansas City','Houston','San Francisco Bay Area','Seattle','Los Angeles','Dallas–Fort Worth',
             'Atlanta','Washington, D.C.','Philadelphia','New York City','Miami–Fort Lauderdale','Milwaukee','Chicago',
            'St. Louis','Pittsburgh','Cincinnati','Los Angeles','Denver','Phoenix','San Francisco Bay Area','San Diego']
    r=0
    for i in mlb_df["team"]:
        tmp_dict[i]=tmp_lst[r]
        r+=1
    mlb_df=mlb_df.astype({"W": int,"L":int})
    mlb_df['WL_Ratio'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    mlb_df['Metropolitan area']= mlb_df['team'].map(tmp_dict)
    
    mlb_out = mlb_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    mlb_out = pd.merge(mlb_out, cities, how = 'left', left_index = True, right_index = True)
    mlb_out['Population'] = mlb_out['Population'].astype(int)
    mlb_out=mlb_out[['WL_Ratio', 'Population']]
    
    #========================================nfl
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df=nfl_df[nfl_df["year"]==2018].dropna()
    nfl_df['team'] = nfl_df['team'].str.replace('*', '', regex = True)
    nfl_df['team'] = nfl_df['team'].str.replace('+', '', regex = True)
    nfl_df=nfl_df[nfl_df.apply(lambda x: x.values.tolist() not
                               in nfl_df[nfl_df["W"].str.contains(r"[A-Za-z]")].values.tolist(), axis=1)]
    nfl_dct = {'New England Patriots': 'Boston',
                      'Miami Dolphins': 'Miami–Fort Lauderdale',
                      'Buffalo Bills': 'Buffalo',
                      'New York Jets': 'New York City',
                      'Baltimore Ravens': 'Baltimore',
                      'Pittsburgh Steelers': 'Pittsburgh',
                      'Cleveland Browns': 'Cleveland',
                      'Cincinnati Bengals': 'Cincinnati',
                      'Houston Texans': 'Houston',
                      'Indianapolis Colts': 'Indianapolis',
                      'Tennessee Titans': 'Nashville',
                      'Jacksonville Jaguars': 'Jacksonville',
                      'Kansas City Chiefs': 'Kansas City',
                      'Los Angeles Chargers': 'Los Angeles',
                      'Denver Broncos': 'Denver',
                      'Oakland Raiders': 'San Francisco Bay Area',
                      'Dallas Cowboys': 'Dallas–Fort Worth',
                      'Philadelphia Eagles': 'Philadelphia',
                      'Washington Redskins': 'Washington, D.C.',
                      'New York Giants': 'New York City',
                      'Chicago Bears': 'Chicago',
                      'Minnesota Vikings': 'Minneapolis–Saint Paul',
                      'Green Bay Packers': 'Green Bay',
                      'Detroit Lions': 'Detroit',
                      'New Orleans Saints': 'New Orleans',
                      'Carolina Panthers': 'Charlotte',
                      'Atlanta Falcons': 'Atlanta',
                      'Tampa Bay Buccaneers': 'Tampa Bay Area',
                      'Los Angeles Rams': 'Los Angeles',
                      'Seattle Seahawks': 'Seattle',
                      'San Francisco 49ers': 'San Francisco Bay Area',
                      'Arizona Cardinals': 'Phoenix'}

    nfl_df['Metropolitan area'] = nfl_df['team'].map(nfl_dct)
    nfl_df=nfl_df.astype({"W": int,"L":int})
    nfl_df['WL_Ratio'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    
    nfl_out = nfl_df[['WL_Ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    cities = cities.set_index('Metropolitan area')
    cities.rename(columns={"Population (2016 est.)[8]":"Population"},inplace=True)
    nfl_out = pd.merge(nfl_out, cities, how = 'left', left_index = True, right_index = True)
    nfl_out['Population'] = nfl_out['Population'].astype(int)
    nfl_out=nfl_out[['WL_Ratio', 'Population']]
    
    #============final
    
    final_output = nhl_out.rename(columns = {'WL_Ratio': 'WL_Ratio_NHL', 'Population': 'Population_NHL'})
    
    final_output['NHL'] = final_output['Population_NHL'].apply(lambda x: 1 if x>0 else 0)
    final_output = pd.merge(final_output, 
                            nba_out, 
                            how = 'outer', 
                            left_index = True, 
                            right_index = True).rename(columns = {'WL_Ratio': 'WL_Ratio_NBA', 
                                                                  'Population': 'Population_NBA'})
 
    final_output['NBA'] = final_output['Population_NBA'].apply(lambda x: 1 if x>0 else 0)
    
    final_output = pd.merge(final_output, 
                            mlb_out, 
                            how = 'outer', 
                            left_index = True, 
                            right_index = True).rename(columns = {'WL_Ratio': 'WL_Ratio_MLB', 
                                                                  'Population': 'Population_MLB'})
    
    final_output['MLB'] = final_output['Population_MLB'].apply(lambda x: 1 if x>0 else 0)
    
    
    final_output = pd.merge(final_output, 
                            nfl_out, 
                            how = 'outer', 
                            left_index = True, 
                            right_index = True).rename(columns = {'WL_Ratio': 'WL_Ratio_NFL', 
                                                                  'Population': 'Population_NFL'})
    final_output['NFL'] = final_output['Population_NFL'].apply(lambda x: 1 if x>0 else 0)
    for sport1 in sports:
        for sport2 in sports:
            if sport1!=sport2:
                test_pairs = final_output[(final_output[sport1] == 1) & (final_output[sport2] == 1)]
                p_values.loc[sport1, sport2] = stats.ttest_rel(test_pairs['WL_Ratio_{}'.format(sport1)], 
                                                               test_pairs['WL_Ratio_{}'.format(sport2)])[1]
    
    #assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    #assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
